# POST CODE CSV WRANGLING

In [2]:
import pandas as pd

In [2]:
postcode = pd.read_csv('australian_postcodes.csv')

In [3]:
postcode_vic = postcode.copy()
postcode_vic

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,MMM_2015,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,electorate,electoraterating
0,230,200,ANU,ACT,149.119000,-35.277700,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,N2,NaN,NaN,NaN,Durack,NaN
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,...,1.0,1.0,NaN,NaN,N2,NaN,NaN,NaN,Durack,NaN
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,...,2.0,2.0,NaN,NaN,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
3,24049,800,DARWIN CITY,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,...,2.0,2.0,NaN,NaN,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
4,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,...,2.0,2.0,NaN,NaN,NT1,PHN701,NaN,NaN,Lingiari,Rural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,11186,9013,BRISBANE,QLD,152.823141,-27.603479,CITY DC - BRISBANE,LVR,Updated 25-Mar-2020 SA3,30504.0,...,1.0,1.0,NaN,44.349792,Q1,PHN301,NaN,Brisbane,Griffith,NaN
18438,11187,9015,BRISBANE,QLD,152.823141,-27.603479,CITY DC - BRISBANE,LVR,Updated 25-Mar-2020 SA3,30504.0,...,1.0,1.0,NaN,44.349792,Q1,PHN301,NaN,Brisbane,Griffith,NaN
18439,11196,9464,NORTHGATE MC,QLD,153.074982,-27.397055,NaN,NaN,Updated 25-Mar-2020 SA3,30203.0,...,1.0,1.0,NaN,NaN,Q1,PHN301,NaN,Brisbane,Griffith,NaN
18440,11197,9726,GOLD COAST MC,QLD,153.412197,-28.008783,NaN,NaN,Updated 25-Mar-2020 SA3,30910.0,...,1.0,1.0,Moncrieff,NaN,Q1,PHN303,NaN,Gold Coast,McPherson,NaN


In [4]:
postcode_vic = postcode_vic.loc[postcode_vic['state'] == 'VIC']

In [5]:
postcode_vic.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'electorate', 'electoraterating'],
      dtype='object')

In [109]:
postcode_vic = postcode_vic.round({'long': 2, 'lat':2})

In [6]:
# Unique number of unique localities in victorian state
len(list(postcode_vic.locality.unique()))

3340

In [7]:
output = postcode_vic[['lat', 'long', 'locality']].drop_duplicates(['lat', 'long'])

In [18]:
output = output.reset_index()
#output = output.drop('index', axis = 1)

In [20]:
output = output.rename(columns={"index":"locality_id"})

In [27]:
# Write to file
output.to_csv('vic_locality.csv', index=False)

# API CALLS: Weatherbit.io

In [75]:
# API variables
from datetime import datetime as dt
from datetime import timedelta as td
import requests
import json
import pandas as pd
import time

# starting a rewuest session
request_session = requests.Session()

# Date range
end_date = (dt.date(dt.now()) - td(days=1)).strftime("%Y-%m-%d")
start_date = (dt.date(dt.now()) - td(days=5)).strftime("%Y-%m-%d")

# API Keys
api_key1 = 'be7dc4fb6df245869fdb3f50476ceeaf' #Easwar's key
api_key2 = 'aa747feb68304fe3a646e3e4b9aa8352' # My key (cmng16)
api_key3 = 'b87871f1310f48a98e81146f9a39f219' #Alt key (big_bean2022)

In [76]:
end_date

'2022-08-04'

In [142]:
start_date

'2022-07-31'

In [82]:
output = pd.read_csv('vic_locality.csv')

In [88]:
weather_dict = dict()
for i in range(0,output.shape[0]):
    # Calls Latitude and Longitude for a given postcode
    lat = output.loc[i,"lat"]
    lon = output.loc[i,"long"]
    loc_id = output.loc[i, "locality"]
    # API Call
    uv_url = "https://api.weatherbit.io/v2.0/history/daily?lat=" + str(lat) + "&lon=" + str(lon) + '&start_date=' + start_date + '&end_date=' + end_date + '&key=' + api_key2
    # the result from the get request
    result = request_session.get(uv_url)
    print(result.text)
    # weather dict if no data found at given locaiton, NA is established - converts to dictionary value format
    if len(result.text) == 0:
        weather_dict[loc_id] = 'NA'
    else:
        weather_data = json.loads(result.text)
        weather_dict[loc_id] = weather_data
        del weather_dict[loc_id]['sources']

{"timezone":"Australia\/Melbourne","state_code":"07","country_code":"AU","lat":-37.817403,"lon":144.956776,"city_name":"Docklands","station_id":"959360-99999","data":[{"rh":54.8,"max_wind_spd_ts":1659232800,"t_ghi":3233.2,"max_wind_spd":8.8,"solar_rad":80.2,"wind_gust_spd":8.8,"max_temp_ts":1659232800,"min_temp_ts":1659189600,"clouds":77,"max_dni":786.2,"precip_gpm":5.7,"wind_spd":5.3,"slp":1013.3,"ts":1659189600,"max_ghi":525.2,"temp":11.9,"pres":1012.4,"dni":261,"dewpt":2.8,"snow":0,"dhi":30.8,"precip":5.7,"wind_dir":273,"max_dhi":95.9,"ghi":134.7,"max_temp":15,"t_dni":6263.9,"max_uv":2.3,"t_dhi":739.5,"datetime":"2022-07-31","t_solar_rad":1924.1,"min_temp":10.6,"max_wind_dir":273,"snow_depth":null},{"rh":66.6,"max_wind_spd_ts":1659319200,"t_ghi":3268.7,"max_wind_spd":7.2,"solar_rad":92.8,"wind_gust_spd":7.2,"max_temp_ts":1659326400,"min_temp_ts":1659304800,"clouds":65,"max_dni":788.1,"precip_gpm":2.1,"wind_spd":4.3,"slp":1011.6,"ts":1659276000,"max_ghi":529.5,"temp":11.8,"pres":1010

In [324]:
# Remoes outer key
all_data = []
for key in weather_dict.keys():
    all_data.append(weather_dict[key])

In [338]:
all_data_dropna = [x for x in all_data if str(x) != 'NA']

In [351]:
# Json normalises outer list
csv1 = pd.json_normalize(all_data_dropna)

In [353]:
# Saves current CSV table
csv1.to_csv("raw_dropped.csv")

In [432]:
# Generates tables for embedded data
for i in range(0,len(all_data_dropna)):
    if i == 0:
        details = pd.DataFrame(all_data_dropna[i]['data'])
        details['lat'] = all_data_dropna[i]['lat']
        details['lon'] = all_data_dropna[i]['lon']
        df = details
    else:
        details = pd.DataFrame(all_data_dropna[i]['data'])
        details['lat'] = all_data_dropna[i]['lat']
        details['lon'] = all_data_dropna[i]['lon']
        df = pd.concat([df, details], axis = 0)

In [435]:
all_drop_df = pd.json_normalize(all_data_dropna)

In [436]:
all_drop_df.columns

Index(['timezone', 'state_code', 'country_code', 'lat', 'lon', 'city_name',
       'station_id', 'data', 'city_id'],
      dtype='object')

In [437]:
df.columns

Index(['rh', 'max_wind_spd_ts', 't_ghi', 'max_wind_spd', 'solar_rad',
       'wind_gust_spd', 'max_temp_ts', 'min_temp_ts', 'clouds', 'max_dni',
       'precip_gpm', 'wind_spd', 'slp', 'ts', 'max_ghi', 'temp', 'pres', 'dni',
       'dewpt', 'snow', 'dhi', 'precip', 'wind_dir', 'max_dhi', 'ghi',
       'max_temp', 't_dni', 'max_uv', 't_dhi', 'datetime', 't_solar_rad',
       'min_temp', 'max_wind_dir', 'snow_depth', 'lat', 'lon'],
      dtype='object')

In [438]:
output_df = pd.merge(all_drop_df, df, left_on = ["lat", 'lon'], right_on= ["lat", 'lon'])

In [444]:
output_df1 = pd.merge(output_df, output, left_on = ["lat", "lon"], right_on=["lat", "long"])

In [446]:
output_df1.to_csv('raw_wb_api_data.csv')

In [449]:
# Simplified dataset of api data
size = df.groupby(['lat', 'lon']).size()

In [423]:
output_df.shape

(6584, 43)

In [396]:
output_df1 = pd.merge(output_df, output, left_on=['lat','lon'], right_on=['lat','long'])

In [29]:
def uv_level_gauge(row):
    if row['max_uv'] >= 1 and row['max_uv'] < 3:
        uv_level = "LOW"
    elif row['max_uv'] >= 3 and row['max_uv'] < 6:
        uv_level = "MODERATE"
    elif row['max_uv'] >= 6 and row['max_uv'] < 9:
        uv_level = "HIGH"
    elif row['max_uv'] >= 9 and row['max_uv'] < 11:
        uv_level = "VERY HIGH"
    elif row['max_uv'] >= 11:
        uv_level = "EXTREME"
    else:
        uv_level = "NULL"
    return uv_level

def uv_id_gauge(row):
    if row['max_uv'] >= 1 and row['max_uv'] < 3:
        uv_id = 0
    elif row['max_uv'] >= 3 and row['max_uv'] < 6:
        uv_id = 1
    elif row['max_uv'] >= 6 and row['max_uv'] < 9:
        uv_id = 2
    elif row['max_uv'] >= 9 and row['max_uv'] < 11:
        uv_id = 3
    elif row['max_uv'] >= 11:
        uv_id = 4
    else:
        uv_id = 5
    return uv_id
# Adds uv_leves and uv_id columns based off the max_uv
output_df1["uv_levels"] =  output_df1.apply(uv_level_gauge, axis = 1)
output_df1["uv_id"] =  output_df1.apply(uv_id_gauge, axis = 1)

In [30]:
output_df1.to_csv("raw_wb_api_data.csv")

In [31]:
# Drops undesired columns
output_df2 = output_df1[["locality", "lat","lon", "locality_id", "city_id", "city_name", "max_uv", "datetime", "t_solar_rad", "uv_levels", "uv_id"]]

In [5]:
import pandas as pd

In [27]:
output_df2 = pd.read_csv("weatherbit.io API/raw_wb_api_data.csv")

In [32]:
output_df2

,locality,lat,lon,locality_id,city_id,city_name,max_uv,datetime,t_solar_rad,uv_levels,uv_id
0,EAST MELBOURNE,-37.818517,144.982207,1,6952201,East Melbourne,2.3,2022-07-31,1923.4,LOW,0
1,EAST MELBOURNE,-37.818517,144.982207,1,6952201,East Melbourne,2.3,2022-08-01,2226.5,LOW,0
2,EAST MELBOURNE,-37.818517,144.982207,1,6952201,East Melbourne,3.5,2022-08-03,3340.8,MODERATE,1
3,EAST MELBOURNE,-37.818517,144.982207,1,6952201,East Melbourne,2.7,2022-08-02,2698.5,LOW,0
4,WEST MELBOURNE,-37.810871,144.949592,2,7281818,West Melbourne,2.3,2022-07-31,1924.1,LOW,0
...,...,...,...,...,...,...,...,...,...,...,...
1475,BALLYROGAN,-37.424288,143.143621,374,2177756,Ararat,3.6,2022-08-03,3372.3,MODERATE,1
1476,BULGANA,-37.129753,142.994830,375,2177756,Ararat,1.6,2022-08-02,1400.9,LOW,0
1477,BULGANA,-37.129753,142.994830,375,2177756,Ararat,1.5,2022-07-31,1697.4,LOW,0
1478,BULGANA,-37.129753,142.994830,375,2177756,Ararat,3.5,2022-08-03,3305.2,MODERATE,1


In [33]:
# Saves the simplified output
output_df2.to_csv("simplified_wb_api_data.csv")

# OBSELETE

In [94]:
# Saves dictionary to a json format
with open('raw_weather_data.json', 'w') as f:
    json.dump(weather_dict, f, ensure_ascii=False, indent=4)

In [240]:
# Loads modified csv from excel data
raw_data = pd.read_csv('raw_data_pivot.csv')

In [245]:
data_drop = raw_data.dropna()

In [246]:
uv_data = data_drop[["data"]]

In [262]:
uv_data = json.loads(uv_data.to_json())

In [282]:
len(uv_data['data'])

370

In [289]:
count = 1

In [315]:
temp = uv_data['data'][str(count)]

In [316]:
temp

"[{'rh': 54.8, 'max_wind_spd_ts': 1659232800, 't_ghi': 3232.1, 'max_wind_spd': 8.8, 'solar_rad': 80.1, 'wind_gust_spd': 8.8, 'max_temp_ts': 1659232800, 'min_temp_ts': 1659189600, 'clouds': 77, 'max_dni': 786.4, 'precip_gpm': 5.7, 'wind_spd': 5.3, 'slp': 1013.3, 'ts': 1659189600, 'max_ghi': 525.1, 'temp': 11.9, 'pres': 1012.4, 'dni': 261.1, 'dewpt': 2.8, 'snow': 0, 'dhi': 30.8, 'precip': 5.7, 'wind_dir': 273, 'max_dhi': 95.8, 'ghi': 134.7, 'max_temp': 15, 't_dni': 6265.4, 'max_uv': 2.3, 't_dhi': 738.9, 'datetime': '2022-07-31', 't_solar_rad': 1923.4, 'min_temp': 10.6, 'max_wind_dir': 273, 'snow_depth': None}, {'rh': 66.6, 'max_wind_spd_ts': 1659319200, 't_ghi': 3267.6, 'max_wind_spd': 7.2, 'solar_rad': 92.8, 'wind_gust_spd': 7.2, 'max_temp_ts': 1659326400, 'min_temp_ts': 1659304800, 'clouds': 65, 'max_dni': 788.3, 'precip_gpm': 2.1, 'wind_spd': 4.3, 'slp': 1011.6, 'ts': 1659276000, 'max_ghi': 529.4, 'temp': 11.8, 'pres': 1010.7, 'dni': 262.5, 'dewpt': 5.6, 'snow': 0, 'dhi': 31, 'precip'

In [317]:
json.loads(temp)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [292]:
pd.json_normalize(uv_data['data'][str(count)])

NotImplementedError: 

In [215]:
data_drop.shape

(370, 10)

In [239]:
json.loads(str(data_drop.loc[1,['data']].values))

["[{'rh': 54.8, 'max_wind_spd_ts': 1659232800, 't_ghi': 3232.1, 'max_wind_spd': 8.8, 'solar_rad': 80.1, 'wind_gust_spd': 8.8, 'max_temp_ts': 1659232800, 'min_temp_ts': 1659189600, 'clouds': 77, 'max_dni': 786.4, 'precip_gpm': 5.7, 'wind_spd': 5.3, 'slp': 1013.3, 'ts': 1659189600, 'max_ghi': 525.1, 'temp': 11.9, 'pres': 1012.4, 'dni': 261.1, 'dewpt': 2.8, 'snow': 0, 'dhi': 30.8, 'precip': 5.7, 'wind_dir': 273, 'max_dhi': 95.8, 'ghi': 134.7, 'max_temp': 15, 't_dni': 6265.4, 'max_uv': 2.3, 't_dhi': 738.9, 'datetime': '2022-07-31', 't_solar_rad': 1923.4, 'min_temp': 10.6, 'max_wind_dir': 273, 'snow_depth': None}, {'rh': 66.6, 'max_wind_spd_ts': 1659319200, 't_ghi': 3267.6, 'max_wind_spd': 7.2, 'solar_rad': 92.8, 'wind_gust_spd': 7.2, 'max_temp_ts': 1659326400, 'min_temp_ts': 1659304800, 'clouds': 65, 'max_dni': 788.3, 'precip_gpm': 2.1, 'wind_spd': 4.3, 'slp': 1011.6, 'ts': 1659276000, 'max_ghi': 529.4, 'temp': 11.8, 'pres': 1010.7, 'dni': 262.5, 'dewpt': 5.6, 'snow': 0, 'dhi': 31, 'precip

In [222]:
for i in range(1, data_drop.shape[0]+1):
    pd.DataFrame(data_drop.loc[i,['data']])

KeyError: 235

In [204]:
pd.DataFrame.from_dict(data_drop.loc['data',0])

KeyError: 0

In [185]:
pivot = raw_data.melt(id_vars=['features'], value_vars=)

KeyError: "The following 'id_vars' are not present in the DataFrame: ['features']"

In [140]:
output.shape

(1003, 4)

# API CALL: openWeather.io

In [28]:
# API variables
from datetime import datetime as dt
from datetime import timedelta as td
import requests
import json
import pandas as pd
import time

# starting a rewuest session
request_session = requests.Session()

# Date range
end_date = (dt.date(dt.now()) - td(days=1)).strftime("%Y-%m-%d")
start_date = (dt.date(dt.now()) - td(days=31)).strftime("%Y-%m-%d")

# API Keys
ow_api_key = 'ccbbe692467b0604c46057286da06f8b' # (cmng16)

In [35]:
# Process of tabulising raw data
for i in range(0,30):
    timestamp = (dt.date(dt.now()) - td(days=i)).strftime("%Y-%m-%d")
    ow_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={ts}&appid={api_key}".format(lat = lat, lon = lon, ts = timestamp, api_key = ow_api_key)

In [36]:
ow_url

'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=-37.817403&lon=144.956776&dt=2022-07-06&appid=ccbbe692467b0604c46057286da06f8b'